In [1]:
import os
import math
import random
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
import time

# Set precision
getcontext().prec = 30
scale_factor = Decimal("1")

class ChristmasTree:
    def __init__(self, center_x="0", center_y="0", angle="0"):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal("0.15")
        trunk_h = Decimal("0.2")
        base_w = Decimal("0.7")
        mid_w  = Decimal("0.4")
        top_w  = Decimal("0.25")
        tip_y = Decimal("0.8")
        tier_1_y = Decimal("0.5")
        tier_2_y = Decimal("0.25")
        base_y = Decimal("0.0")
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon(
            [
                (Decimal("0.0") * scale_factor, tip_y * scale_factor),
                (top_w / Decimal("2") * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal("4") * scale_factor, tier_1_y * scale_factor),
                (mid_w / Decimal("2") * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal("4") * scale_factor, tier_2_y * scale_factor),
                (base_w / Decimal("2") * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal("2") * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal("2") * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal("2")) * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal("2")) * scale_factor, base_y * scale_factor),
                (-(base_w / Decimal("2")) * scale_factor, base_y * scale_factor),
                (-(mid_w / Decimal("4")) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal("2")) * scale_factor, tier_2_y * scale_factor),
                (-(top_w / Decimal("4")) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal("2")) * scale_factor, tier_1_y * scale_factor),
            ]
        )

        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(
            rotated,
            xoff=float(self.center_x * scale_factor),
            yoff=float(self.center_y * scale_factor),
        )

    def clone(self):
        return ChristmasTree(center_x=str(self.center_x), center_y=str(self.center_y), angle=str(self.angle))

def parse_csv(csv_path):
    df = pd.read_csv(csv_path)
    df["x"] = df["x"].astype(str).str.strip().str.lstrip("s")
    df["y"] = df["y"].astype(str).str.strip().str.lstrip("s")
    df["deg"] = df["deg"].astype(str).str.strip().str.lstrip("s")
    df[["group_id", "item_id"]] = df["id"].str.split("_", n=2, expand=True)
    
    dict_of_tree_list = {}
    for group_id, group_data in df.groupby("group_id"):
        tree_list = [
            ChristmasTree(center_x=row["x"], center_y=row["y"], angle=row["deg"])
            for _, row in group_data.iterrows()
        ]
        dict_of_tree_list[int(group_id)] = tree_list
    return dict_of_tree_list

def write_submission(dict_of_tree_list, out_file):
    rows = []
    sorted_keys = sorted(dict_of_tree_list.keys())
    for group_name in sorted_keys:
        tree_list = dict_of_tree_list[group_name]
        for item_id, tree in enumerate(tree_list):
            rows.append(
                {
                    "id": f"{group_name}_{item_id}",
                    "x": f"s{tree.center_x}",
                    "y": f"s{tree.center_y}",
                    "deg": f"s{tree.angle}",
                }
            )
    pd.DataFrame(rows).to_csv(out_file, index=False)

def get_bbox_side(trees):
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1])

def has_overlap(trees):
    polys = [t.polygon for t in trees]
    tree = STRtree(polys)
    for i, poly in enumerate(polys):
        # query returns indices of geometries that intersect
        # we need to check if any intersection is not with itself
        indices = tree.query(poly)
        for idx in indices:
            if idx != i:
                if poly.intersection(polys[idx]).area > 1e-9: # Tolerance
                    return True
    return False

def simulated_annealing(trees, n_iter=1000, initial_temp=1.0, cooling_rate=0.995):
    current_trees = [t.clone() for t in trees]
    current_side = get_bbox_side(current_trees)
    best_trees = [t.clone() for t in current_trees]
    best_side = current_side
    
    temp = initial_temp
    
    for i in range(n_iter):
        # Create candidate by perturbing one tree
        candidate_trees = [t.clone() for t in current_trees]
        idx = random.randint(0, len(candidate_trees) - 1)
        
        # Perturbation: move towards center or random small shift
        # Bias towards center (0,0) to compact
        # But center of bbox is not necessarily (0,0). 
        # Let's just do random small shift for now.
        
        shift_scale = temp * 0.5 # Decay shift size with temp
        dx = random.uniform(-shift_scale, shift_scale)
        dy = random.uniform(-shift_scale, shift_scale)
        d_angle = random.uniform(-10 * temp, 10 * temp)
        
        t = candidate_trees[idx]
        t.center_x += Decimal(dx)
        t.center_y += Decimal(dy)
        t.angle += Decimal(d_angle)
        
        # Re-create polygon
        # Note: This is inefficient, creating new object every time. 
        # But for N=200 it might be slow. For small N it's fine.
        # We need to update the polygon of the moved tree
        # The class creates polygon in __init__, so we need to re-init or update
        # Let's just create a new tree object for simplicity of code, though slow
        candidate_trees[idx] = ChristmasTree(t.center_x, t.center_y, t.angle)
        
        # Check constraints
        if has_overlap(candidate_trees):
            # Invalid state, reject immediately (hard constraint)
            # Or we could use penalty in energy, but hard constraint is safer for validity
            continue
            
        candidate_side = get_bbox_side(candidate_trees)
        
        # Metropolis acceptance
        delta = candidate_side - current_side
        if delta < 0 or random.random() < math.exp(-delta / temp):
            current_trees = candidate_trees
            current_side = candidate_side
            
            if current_side < best_side:
                best_side = current_side
                best_trees = [t.clone() for t in current_trees]
                print(f"Iter {i}: New best side {best_side}")
        
        temp *= cooling_rate
        
    return best_trees, best_side

def main():
    print("Loading submission...")
    tree_dict = parse_csv("submission.csv")
    
    # Select Ns to optimize. 
    # Based on analysis, N=1, 2, 3 have high gaps.
    # Let's try N=1..5 first as a test.
    target_ns = [1, 2, 3, 4, 5]
    
    improved_dict = tree_dict.copy()
    
    for n in target_ns:
        if n not in tree_dict: continue
        print(f"Optimizing N={n}...")
        trees = tree_dict[n]
        
        # Run SA
        # Increase iterations for real run
        best_trees, best_side = simulated_annealing(trees, n_iter=500, initial_temp=0.5, cooling_rate=0.99)
        
        improved_dict[n] = best_trees
        print(f"N={n} Final Side: {best_side}")
        
    print("Writing submission...")
    write_submission(improved_dict, "submission_sa.csv")

if __name__ == "__main__":
    main()


Loading submission...


Optimizing N=1...
N=1 Final Side: 0.8131727983645263
Optimizing N=2...


N=2 Final Side: 0.9495042736742613
Optimizing N=3...


N=3 Final Side: 1.1420312680063256
Optimizing N=4...


N=4 Final Side: 1.2908057151967518
Optimizing N=5...


N=5 Final Side: 1.443692472208735
Writing submission...
